# 1) Initialization


In [ ]:
import sys
# Set the project path here
PathProj = "C:\\Users\\tepang\\Desktop\\Test3"
sys.path.append(PathProj)

from Calibration import *

In [ ]:
PPdispatchError

In [ ]:
set_period(t_from=PERIOD_FROM, t_to=PERIOD_TO)
set_scenario({'sys demand': 'Baseline'})

Results = solve(simulation_name='Baseline', runchecks=False, daily_progress=False)

# 2) Running Calibration Module (if required)
- skip this step if user is satisfied with the results generated 

<h3><i><font color=#117A65>2.1) Pre-Calibration</font></i></h3>

In [ ]:
# 1) Run Pre-Calibration, need at least 2 runs to get a valid std result

prepare_save_file(path.join(PATHS['Proj'],'Scripts\\In_Progress\\pre-calibration_in_progress.csv'))
Pre_Calibration(2500)

<h3><i><font color=#117A65>2.2) Normalize Parameters</font></i></h3>

In [ ]:
# 2) Normalize Parameters

# Load the Pre-Calibration file
filename=path.join(PATHS['Proj'],'Scripts\\In_Progress\\pre-calibration_in_progress.csv')
data = pd.read_csv(filename, index_col=[0])
data.to_csv(path.join(PATHS['Proj'],'Scripts\\pre-calibration.csv'), index=True)
print('Pre-Calibration file transferred complete')

# PRE_CALIBRATE_PARAMS = pd.read_csv(path.join(PATHS['Proj'],'Scripts\\pre-calibration.csv'),index_col=[0]) 
# Use PRE_CALIBRATE_PARAMS to calculate normalize values

data.describe()

<h3><i><font color=#117A65>2.3) Calibration</font></i></h3>

In [ ]:
# 3) Run Calibration
# Need to calculate normalize values before running Calibration
max_evals = 500

File_in_progress = path.join(PATHS['Proj'],'Scripts\\In_Progress\\calibration_in_progress.csv')
prepare_save_file(File_in_progress)
best, PARAMS_DFS, RANDSEEDS_DFS  = Calibration(max_evals,File_in_progress)


In [ ]:
# Display Best Score
# display_run_score(RUN) # RUN is the run no. in excel
# display_run_score(trials.best_trial['tid']+1)
# best = trials.best_trial['tid']+1
display_run_score(best)

In [ ]:
# Running with Best Parameters
import gd_core as __core

PPdb['params'] = PARAMS_DFS[best-1].copy()
PPdb['randseeds'] = RANDSEEDS_DFS[best-1].copy()

__core.reinit_fleet()

Results = solve(simulation_name='Baseline', runchecks=False, daily_progress=False)

# 3) Visualization of Results

In [ ]:
# Online capacities (baseline)
plot_OnlineCapacity(Results, ppclasses='default') #ppclasses='default','total'

In [ ]:
plot_fuelmix(Results,)

# 4) Generation of AH required for WRF input

In [ ]:
calc_allheat(Results)
compiled_prms = thermal_analysis(GenFleet, dParameters)

calc_heatstreams1(GenFleet, Results, by='outlet',)
calc_heatstreams1(GenFleet, Results, by='kind', latentheatfactors=dParameters['latent heat factors'])

In [ ]:
# Prepare the WRF input files for a day
# unit options 'W/m^2' 'MW'

WRF_SH, WRF_LH, WRF_Sea, total_MWh = prep_WRF_inputs(
    scenario='Test WRF - 2016 Apr (3)', 
    GenFleet=GenFleet, 
    PPcells_only=False,   # Set to False if you want the full grid (sparse)
    With_height=True,
    unit='MW',
    day='2016 Apr 15', 
    write_to_disk = True
)